$$ \LaTeX \text{ command declarations here.}
\newcommand{\R}{\mathbb{R}}
\renewcommand{\vec}[1]{\mathbf{#1}}
\newcommand{\X}{\mathcal{X}}
\newcommand{\D}{\mathcal{D}}
\newcommand{\G}{\mathcal{G}}
\newcommand{\L}{\mathcal{L}}
\newcommand{\X}{\mathcal{X}}
\newcommand{\Parents}{\mathrm{Parents}}
\newcommand{\NonDesc}{\mathrm{NonDesc}}
\newcommand{\I}{\mathcal{I}}
\newcommand{\dsep}{\text{d-sep}}
\newcommand{\Cat}{\mathrm{Categorical}}
\newcommand{\Bin}{\mathrm{Binomial}}
$$

# HMMs and the Baum-Welch Algorithm

As covered in lecture, the Baum-Welch Algorithm is a derivation of the EM algorithm for HMMs where we learn the paramaters A, B and $\pi$ given a set of observations.

In this hands-on exercise we will **build upon the forward and backward algorithms from last exercise**, which can be used for the E-step, and implement Baum-Welch ourselves!

Like last time, we'll work with an example where we observe a sequence of words backed by a latent part of speech variable.

$X$: discrete distribution over bag of words

$Z$: discrete distribution over parts of speech

$A$: the probability of a part of speech given a previous part of speech, e.g, what do we expect to see after a noun? 

$B$: the distribution of words given a particular part of speech, e.g, what words are we likely to see if we know it is a verb?

$x_{i}s$ a sequence of observed words (a sentence). Note: in for both variables we have a special "end" outcome that signals the end of a sentence. This makes sense as a part of speech tagger would like to have a sense of sentence boundaries.


## Review: Forward / Backward

Here are solutions to last hands-on lecture's coding problems along with example uses with a pre-defined A and B matrices.

In [1]:
import numpy as np
np.set_printoptions(suppress=True)

parts_of_speech = DETERMINER, NOUN, VERB, END = 0, 1, 2, 3
words = THE, DOG, CAT, WALKED, RAN, IN, PARK, END = 0, 1, 2, 3, 4, 5, 6, 7

# transition probabilities
A = np.array([
        # D     N   V   E
        [0.1, 0.8, 0.1, 0.0],  # D: determiner most likely to go to noun
        [0.1, 0.1, 0.6, 0.2],  # N: noun most likely to go to verb
        [0.4, 0.3, 0.2, 0.1],  # V 
        [0.0, 0.0, 0.0, 1.0]]) # E: end always goes to end

# distribution of parts of speech for the first word of a sentence
pi = np.array([0.4, 0.3, 0.3, 0.0])

# emission probabilities
B = np.array([
        # D     N     V     E
        [ 0.8,  0.1,  0.1,  0. ],  # the
        [ 0.1,  0.8,  0.1,  0. ],  # dog
        [ 0.1,  0.8,  0.1,  0. ],  # cat
        [ 0. ,  0. ,  1. ,  0. ],  # walked
        [ 0. ,  0.2 , 0.8 ,  0. ], # ran
        [ 1. ,  0. ,  0. ,  0. ],  # in
        [ 0. ,  0.1,  0.9,  0. ],  # park
        [ 0. ,  0. ,  0. ,  1. ]]) # end


In [2]:
import pandas as pd

pos_labels = ["D", "N", "V", "E"]
word_labels = ["the", "dog", "cat", "walked", "ran", "in", "park", "end"]

def print_B(B):
    print(pd.DataFrame(B, columns=pos_labels, index=word_labels))
    
def print_A(A):
    print(pd.DataFrame(A, columns=pos_labels, index=pos_labels))
        
print_A(A)
print_B(B)

     D    N    V    E
D  0.1  0.8  0.1  0.0
N  0.1  0.1  0.6  0.2
V  0.4  0.3  0.2  0.1
E  0.0  0.0  0.0  1.0
          D    N    V    E
the     0.8  0.1  0.1  0.0
dog     0.1  0.8  0.1  0.0
cat     0.1  0.8  0.1  0.0
walked  0.0  0.0  1.0  0.0
ran     0.0  0.2  0.8  0.0
in      1.0  0.0  0.0  0.0
park    0.0  0.1  0.9  0.0
end     0.0  0.0  0.0  1.0


In [3]:
def forward(params, observations):
    pi, A, B = params
    N = len(observations)
    S = pi.shape[0]
    
    alpha = np.zeros((N, S))
    
    # base case
    alpha[0, :] = pi * B[observations[0], :]
    
    # recursive case
    for i in range(1, N):
        for s2 in range(S):
            for s1 in range(S):
                alpha[i, s2] += alpha[i-1, s1] * A[s1, s2] * B[observations[i], s2]    
    
    return (alpha, np.sum(alpha[N-1,:]))

def print_forward(params, observations):
    alpha, za = forward(params, observations)
    print(pd.DataFrame(
            alpha, 
            columns=pos_labels, 
            index=[word_labels[i] for i in observations]))

print_forward((pi, A, B), [THE, DOG, WALKED, IN, THE, PARK, END])
print_forward((pi, A, B), [THE, CAT, RAN, IN, THE, PARK, END])

               D         N         V        E
the     0.320000  0.030000  0.030000  0.00000
dog     0.004700  0.214400  0.005600  0.00000
walked  0.000000  0.000000  0.130230  0.00000
in      0.052092  0.000000  0.000000  0.00000
the     0.004167  0.004167  0.000521  0.00000
park    0.000000  0.000391  0.002719  0.00000
end     0.000000  0.000000  0.000000  0.00035
             D         N         V         E
the   0.320000  0.030000  0.030000  0.000000
cat   0.004700  0.214400  0.005600  0.000000
ran   0.000000  0.005376  0.104184  0.000000
in    0.042211  0.000000  0.000000  0.000000
the   0.003377  0.003377  0.000422  0.000000
park  0.000000  0.000317  0.002203  0.000000
end   0.000000  0.000000  0.000000  0.000284


In [4]:
def backward(params, observations):
    pi, A, B = params
    N = len(observations)
    S = pi.shape[0]
    
    beta = np.zeros((N, S))
    
    # base case
    beta[N-1, :] = 1
    
    # recursive case
    for i in range(N-2, -1, -1):
        for s1 in range(S):
            for s2 in range(S):
                beta[i, s1] += beta[i+1, s2] * A[s1, s2] * B[observations[i+1], s2]
    
    return (beta, np.sum(pi * B[observations[0], :] * beta[0,:]))

backward((pi, A, B), [THE, DOG, WALKED, IN, THE, PARK, END])

(array([[ 0.00104026,  0.00016397,  0.00040858,  0.        ],
        [ 0.0002688 ,  0.0016128 ,  0.0005376 ,  0.        ],
        [ 0.000672  ,  0.000672  ,  0.002688  ,  0.        ],
        [ 0.00672   ,  0.004     ,  0.01016   ,  0.        ],
        [ 0.025     ,  0.056     ,  0.024     ,  0.        ],
        [ 0.        ,  0.2       ,  0.1       ,  1.        ],
        [ 1.        ,  1.        ,  1.        ,  1.        ]]),
 0.00035005824000000022)

In [5]:
# Some utitlities for tracing our implementation below

def left_pad(i, s):
    return "\n".join(["{}{}".format(' '*i, l) for l in s.split("\n")])

def pad_print(i, s):
    print(left_pad(i, s))
    
def pad_print_args(i, **kwargs):
    pad_print(i, "\n".join(["{}:\n{}".format(k, kwargs[k]) for k in sorted(kwargs.keys())]))    


In [6]:
def baum_welch(training, pi, A, B, iterations, trace=False):
    pi, A, B = np.copy(pi), np.copy(A), np.copy(B)  # take copies, as we modify them
    S = pi.shape[0]

    # iterations of EM
    for it in range(iterations):
        if trace:
            pad_print(0, "for it={} in range(iterations)".format(it))
            pad_print_args(2, A=A, B=B, pi=pi, S=S)
        pi1 = np.zeros_like(pi)
        A1 = np.zeros_like(A)
        B1 = np.zeros_like(B)

        for observations in training:
            if trace:
                pad_print(2, "for observations={} in training".format(observations))

            # compute forward-backward matrices
            alpha, za = forward((pi, A, B), observations)
            beta, zb = backward((pi, A, B), observations)
            if trace:
                pad_print(4, """alpha, za = forward((pi, A, B), observations)\nbeta, zb = backward((pi, A, B), observations)""")
                pad_print_args(4, alpha=alpha, beta=beta, za=za, zb=zb)

            assert abs(za - zb) < 1e-6, "it's badness 10000 if the marginals don't agree ({} vs {})".format(za, zb)

            # M-step here, calculating the frequency of starting state, transitions and (state, obs) pairs
            
            # Update PI
            pi1 += alpha[0, :] * beta[0, :] / za

            if trace:
                pad_print(4, "pi1 += alpha[0, :] * beta[0, :] / za")
                pad_print_args(4, pi1=pi1)
                pad_print(4, "for i in range(0, len(observations)):")
            
            # Update B (transition) matrix
            for i in range(0, len(observations)):
                B1[observations[i], :] += alpha[i, :] * beta[i, :] / za
                if trace:
                    pad_print(6, "B1[observations[{i}], :] += alpha[{i}, :] * beta[{i}, :] / za".format(i=i))
            if trace:
                pad_print_args(4, B1=B1)
                pad_print(4, "for i in range(1, len(observations)):")
                
            # Update A (emission) matrix
            for i in range(1, len(observations)):
                if trace: 
                    pad_print(6, "for s1 in range(S={})".format(S))
                for s1 in range(S):
                    if trace: pad_print(8, "for s2 in range(S={})".format(S))
                    for s2 in range(S):
                        A1[s1, s2] += alpha[i - 1, s1] * A[s1, s2] * B[observations[i], s2] * beta[i, s2] / za
                        if trace: pad_print(10, "A1[{s1}, {s2}] += alpha[{i_1}, {s1}] * A[{s1}, {s2}] * B[observations[{i}], {s2}] * beta[{i}, {s2}] / za".format(s1=s1, s2=s2, i=i, i_1=i-1))
            if trace: pad_print_args(4, A1=A1)

        # normalise pi1, A1, B1
        pi = pi1 / np.sum(pi1)
        for s in range(S):
            A[s, :] = A1[s, :] / np.sum(A1[s, :])
            B[s, :] = B1[s, :] / np.sum(B1[s, :])
        
    return pi, A, B


## Training with examples

Let's try producing updated parameters to our HMM using a few examples

In [7]:
pi2, A2, B2 = baum_welch([
        [THE, DOG, WALKED, IN, THE, PARK, END, END], # END -> END needs at least one transition example
        [THE, DOG, RAN, IN, THE, PARK, END],
        [THE, CAT, WALKED, IN, THE, PARK, END],
        [THE, DOG, RAN, IN, THE, PARK, END]], pi, A, B, 10, trace=False)

print("original A")
print_A(A)

print("updated A")
print_A(A2)

print("\noriginal B")
print_B(B)

print("updated B")
print_B(B2)

print_forward((pi2, A2, B2), [THE, DOG, WALKED, IN, THE, PARK, END])

original A
     D    N    V    E
D  0.1  0.8  0.1  0.0
N  0.1  0.1  0.6  0.2
V  0.4  0.3  0.2  0.1
E  0.0  0.0  0.0  1.0
updated A
     D    N    V    E
D  0.0  1.0  0.0  0.0
N  0.0  0.0  1.0  0.0
V  0.5  0.0  0.0  0.5
E  0.0  0.0  0.0  1.0

original B
          D    N    V    E
the     0.8  0.1  0.1  0.0
dog     0.1  0.8  0.1  0.0
cat     0.1  0.8  0.1  0.0
walked  0.0  0.0  1.0  0.0
ran     0.0  0.2  0.8  0.0
in      1.0  0.0  0.0  0.0
park    0.0  0.1  0.9  0.0
end     0.0  0.0  0.0  1.0
updated B
          D    N    V    E
the     0.5  0.5  0.0  0.0
dog     0.0  1.0  0.0  0.0
cat     0.0  1.0  0.0  0.0
walked  0.0  0.0  1.0  0.0
ran     0.0  0.2  0.8  0.0
in      1.0  0.0  0.0  0.0
park    0.0  0.1  0.9  0.0
end     0.0  0.0  0.0  1.0
           D      N       V        E
the     0.50  0.000  0.0000  0.00000
dog     0.00  0.500  0.0000  0.00000
walked  0.00  0.000  0.5000  0.00000
in      0.25  0.000  0.0000  0.00000
the     0.00  0.125  0.0000  0.00000
park    0.00  0.000  0.1125  